# <center>Shape Data</center>

Raw files do not come prepped for analysis. You must indispensably change the raw data. Below, some simple examples of aggregating raw data into a analytical shape.

Based on the movie lens data (http://grouplens.org/datasets/movielens/), let us identify where men and women most disagree with each other on rating.

In [1]:
# Bring the ratings, users, and movies data from the web
# Ratings
ratings = sqlCtx.createDataFrame(
    pd.read_csv(
        'http://files.grouplens.org/datasets/movielens/ml-100k/u.data',
        sep='\t',
        names=['user_id', 'item_id', 'rating', 'timestamp']))

# Users
users = sqlCtx.createDataFrame(
    pd.read_csv(
        'http://files.grouplens.org/datasets/movielens/ml-100k/u.user',
        sep='|',
        names=['user_id_dim', 'age', 'gender', 'occupation', 'zip']))

# Movies
movies = sqlCtx.createDataFrame(
    pd.read_csv(
        'http://files.grouplens.org/datasets/movielens/ml-100k/u.item',
        sep="|",
        names=[
            col.strip()
            for col in
            '''movie_id_dim | movie_title | release_date | video release date |
                          IMDb URL | unknown | Action | Adventure | Animation |
                          Children's | Comedy | Crime | Documentary | Drama | Fantasy |
                          Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
                          Thriller | War | Western'''.split('|')
        ],
        usecols=['movie_id_dim', 'movie_title']))

# Display a preview of all datasets
display(HTML('<h2>Ratings</h2>'))
display(ratings.limit(5).toPandas())
display(HTML('<h2>Users</h2>'))
display(users.limit(5).toPandas())
display(HTML('<h2>Movies</h2>'))
display(movies.limit(5).toPandas())

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


,user_id_dim,age,gender,occupation,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


,movie_id_dim,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


## Dimension/Fact Join

In [2]:
# Join the dimensional definitions with factual tables to obtain denormalized representation
norm_ratings = sqlCtx.sql(
    '''select a.*, b.*, c.* from {0} a join {1} b join {2} c on a.user_id = b.user_id_dim and a.item_id = movie_id_dim'''.
    format(ratings.reg(), users.reg(), movies.reg()))

# Show a preview
display(norm_ratings.limit(10).toPandas())

,user_id,item_id,rating,timestamp,user_id_dim,age,gender,occupation,zip,movie_id_dim,movie_title
0,1,1,5,874965758,1,24,M,technician,85711,1,Toy Story (1995)
1,2,1,4,888550871,2,53,F,other,94043,1,Toy Story (1995)
2,5,1,4,875635748,5,33,F,other,15213,1,Toy Story (1995)
3,6,1,4,883599478,6,42,M,executive,98101,1,Toy Story (1995)
4,10,1,4,877888877,10,53,M,lawyer,90703,1,Toy Story (1995)
5,13,1,3,882140487,13,47,M,educator,29206,1,Toy Story (1995)
6,15,1,1,879455635,15,49,F,educator,97301,1,Toy Story (1995)
7,16,1,5,877717833,16,21,M,entertainment,10309,1,Toy Story (1995)
8,17,1,4,885272579,17,30,M,programmer,06355,1,Toy Story (1995)
9,18,1,5,880130802,18,35,F,other,37212,1,Toy Story (1995)


## Compute Average

In [3]:
# Let us average the rating per movie per gender
avg_ratings = norm_ratings.groupBy(
    ['movie_title', 'gender']).agg(F.avg('rating').alias('rating'))

# Display the aggregated average ratings
display(avg_ratings.limit(10).toPandas())

,movie_title,gender,rating
0,Toy Story (1995),M,3.909910
1,Toy Story (1995),F,3.789916
2,GoldenEye (1995),M,3.178571
3,GoldenEye (1995),F,3.368421
4,Four Rooms (1995),M,3.108108
5,Four Rooms (1995),F,2.687500
6,Get Shorty (1995),M,3.591463
7,Get Shorty (1995),F,3.400000
8,Copycat (1995),M,3.140625
9,Copycat (1995),F,3.772727


## Pivot

In [4]:
# Pivot to flatten gender averages per movie on a single line
pivot_avg_ratings = avg_ratings.groupBy('movie_title').pivot('gender').agg(
    F.first('rating'))

# Display the pivoted values to show averages
display(pivot_avg_ratings.limit(10).toPandas())

,movie_title,F,M
0,Toy Story (1995),3.789916,3.909910
1,GoldenEye (1995),3.368421,3.178571
2,Four Rooms (1995),2.687500,3.108108
3,Get Shorty (1995),3.400000,3.591463
4,Copycat (1995),3.772727,3.140625
5,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,3.600000,3.571429
6,Twelve Monkeys (1995),3.560976,3.861290
7,Babe (1995),4.050000,3.974843
8,Dead Man Walking (1995),3.927711,3.884259
9,Richard III (1995),3.700000,3.869565


# Find Movies

Movies that are rated very differently by men and women are...

In [5]:
#Compute movies that have the maximum ratings polarity between two genders and display top 10
display(
    sqlCtx.sql(
        'select movie_title, F as female_rating, M as male_rating, (M-F) as diff from {0} order by abs(diff) desc limit 10'.
        format(pivot_avg_ratings.reg())).toPandas())

,movie_title,female_rating,male_rating,diff
0,Delta of Venus (1994),1.0,5.000000,4.000000
1,Two or Three Things I Know About Her (1966),1.0,4.666667,3.666667
2,Paths of Glory (1957),1.0,4.419355,3.419355
3,"Magic Hour, The (1998)",1.0,4.250000,3.250000
4,Killer (Bulletproof Heart) (1994),1.0,4.000000,3.000000
5,Love and Death on Long Island (1997),4.0,1.000000,-3.000000
6,So Dear to My Heart (1949),1.0,4.000000,3.000000
7,Loch Ness (1995),4.0,1.000000,-3.000000
8,"Visitors, The (Visiteurs, Les) (1993)",5.0,2.000000,-3.000000
9,"Lay of the Land, The (1997)",4.0,1.000000,-3.000000


<hr />